In [19]:
%pip install bokeh


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
sc.install_pypi_package('matplotlib')

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import pyspark
# from ipywidgets import interact
import numpy as np
import pandas as pd
from tqdm import tqdm
import pandas_ta as ta

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1649544835656_0014,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df=spark.read.parquet("s3://strategymodeldata/equityprice/emrout/equity_1min_taq.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- TimeBarStart: string (nullable = true)
 |-- OpenBarTime: string (nullable = true)
 |-- OpenBidPrice: double (nullable = true)
 |-- OpenBidSize: integer (nullable = true)
 |-- OpenAskPrice: double (nullable = true)
 |-- OpenAskSize: integer (nullable = true)
 |-- FirstTradeTime: string (nullable = true)
 |-- FirstTradePrice: double (nullable = true)
 |-- FirstTradeSize: integer (nullable = true)
 |-- HighBidTime: string (nullable = true)
 |-- HighBidPrice: double (nullable = true)
 |-- HighBidSize: integer (nullable = true)
 |-- HighAskTime: string (nullable = true)
 |-- HighAskPrice: double (nullable = true)
 |-- HighAskSize: integer (nullable = true)
 |-- HighTradeTime: string (nullable = true)
 |-- HighTradePrice: double (nullable = true)
 |-- HighTradeSize: integer (nullable = true)
 |-- LowBidTime: string (nullable = true)
 |-- LowBidPrice: double (nullable = true)
 |-- LowBidSize: integer (nullable = true)
 |-- LowAskTime: string (nullable = true)
 |-- LowAskPrice: doubl

In [7]:
last = df.select("FirstTradePrice","Ticker","datetime")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.select("FirstTradePrice").describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
|summary|  FirstTradePrice|
+-------+-----------------+
|  count|         55944786|
|   mean|201.4394826370669|
| stddev| 381.021394319846|
|    min|           3.4659|
|    max|         5381.925|
+-------+-----------------+

In [32]:
# winTst = Window.partitionBy("Ticker").orderBy("datetime")
# df = df.withColumn('returns1min', (df['FirstTradePrice'] / lag(df["FirstTradePrice"],1).over(winTst))-1)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df.head(1000)

In [4]:
df = df.withColumn('Minute',minute(df.datetime))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
leadWin = Window.partitionBy("Ticker","Date").orderBy("datetime")
df = df.withColumn('FirstTradePrice1minLead',lead(df['FirstTradePrice'],1).over(leadWin))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#schema = StructType([StructField
@pandas_udf('double')
def calculate_return_series(series: pd.Series) -> pd.Series:
    """
    Calculates the return series of a time series.
    The first value will always be NaN.
    Output series retains the index of the input series.
    """
    shifted_series = series.shift(1, axis=0)
    return series / shifted_series - 1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df = df.withColumn('ret1min', calculate_return_series(df.LastTradePrice))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
for i in tqdm(range(2,11)):
    
    @pandas_udf('double')
    def calculate_min_return_series(series: pd.Series) -> pd.Series:
        """
        Calculates the return series of a time series.
        The first value will always be NaN.
        Output series retains the index of the input series.
        """
        shifted_series = series.shift(i, axis=0)
        return series / shifted_series - 1
    
    df = df.withColumn(f'ret{i}min', calculate_min_return_series(df.LastTradePrice))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100%|##########| 9/9 [00:00<00:00, 18.68it/s]

In [68]:
df.select("LastTradePrice","datetime","Ticker","ret1min","ret2min","ret3min","ret4min","ret5min",).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|LastTradePrice|           datetime|Ticker|             ret1min|             ret2min|             ret3min|             ret4min|             ret5min|
+--------------+-------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        168.55|2018-03-22 04:00:00|    FB|                null|                null|                null|                null|                null|
|        168.55|2018-03-22 04:01:00|    FB|                 0.0|                null|                null|                null|                null|
|        168.03|2018-03-22 04:02:00|    FB|-0.00308513794126...|-0.00308513794126...|                null|                null|                null|
|        168.27|2018-03-22 04:03:00|    FB|0.001428316372076...|-0.00166122812221...|-0.00166122812221...|

In [8]:
@pandas_udf('double')
def calculate_fwd_return_series(series: pd.Series) -> pd.Series:
    """
    Calculates the return series of a time series.
    The first value will always be NaN.
    Output series retains the index of the input series.
    """
    shifted_series = series.shift(-1, axis=0)
    return series / shifted_series - 1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# df = df.withColumn('fwd1min', calculate_fwd_return_series(df.ret1min))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:


for i in tqdm(range(1,11)):
    
    df=df.withColumn(f"fwd{i}min",lead(df.ret1min,i).over(Window.partitionBy("Ticker","Date").orderBy("datetime")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100%|##########| 10/10 [00:00<00:00, 21.65it/s]

In [10]:
df.select("LastTradePrice","datetime","Ticker","ret1min","fwd1min").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------------------+------+--------------------+--------------------+
|LastTradePrice|           datetime|Ticker|             ret1min|             fwd1min|
+--------------+-------------------+------+--------------------+--------------------+
|         180.7|2018-03-12 04:00:00|  AAPL|-0.33284105593501945|-0.00654241574578...|
|         180.7|2018-03-12 04:00:00|  AAPL|-0.00654241574578...|0.001660210293303...|
|         181.0|2018-03-12 04:01:00|  AAPL|0.001660210293303...|0.001660210293303...|
|         181.0|2018-03-12 04:01:00|  AAPL|0.001660210293303...|                null|
|          null|2018-03-12 04:02:00|  AAPL|                null|                null|
|          null|2018-03-12 04:02:00|  AAPL|                null|                null|
|         181.0|2018-03-12 04:03:00|  AAPL|                null|                null|
|         181.0|2018-03-12 04:03:00|  AAPL|                null|                null|
|          null|2018-03-12 04:04:00|  AAPL|           

In [11]:
@pandas_udf('double')
def calculate_mfi(high: pd.Series, low: pd.Series, close: pd.Series, volume: pd.Series) -> pd.Series:
    """
    
    """
    mfi = ta.mfi(high,low,close,volume,length=14)
    return mfi

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
@pandas_udf('double')
def calculate_bop(open: pd.Series, high: pd.Series, low: pd.Series, close: pd.Series) -> pd.Series:
    """
    
    """
    bop = (ta.bop(high,low,close,volume,length=14)
    return bop
           
@pandas_udf('double')
def calculate_cci(high: pd.Series, low: pd.Series, close: pd.Series) -> pd.Series:
       """

       """
        cci = ta.cci(high,low,close)
        return cci

@pandas_udf('double')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
invalid syntax (<stdin>, line 7)
  File "<stdin>", line 7
    return bop
         ^
SyntaxError: invalid syntax



In [13]:
df = df.withColumn('mfi', calculate_mfi(df.HighTradePrice,df.LowTradePrice,df.LastTradePrice,df.FinraVolume))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
df.select("LastTradePrice","datetime","Ticker","ret1min","fwd1min","mfi").filter(df.mfi.isNotNull()).show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------------------+------+--------------------+--------------------+------------------+
|LastTradePrice|           datetime|Ticker|             ret1min|             fwd1min|               mfi|
+--------------+-------------------+------+--------------------+--------------------+------------------+
|        180.72|2018-03-12 08:00:00|  AAPL|3.874896208135858...|3.874896208135858...|              null|
|        180.72|2018-03-12 08:00:00|  AAPL|3.874896208135858...|-4.98007968127489...|              null|
|        180.63|2018-03-12 08:01:00|  AAPL|-4.98007968127489...|-4.98007968127489...|              null|
|        180.63|2018-03-12 08:01:00|  AAPL|-4.98007968127489...|                 0.0|              null|
|        180.63|2018-03-12 08:02:00|  AAPL|                 0.0|                 0.0|              null|
|        180.63|2018-03-12 08:02:00|  AAPL|                 0.0|                null|              null|
|          null|2018-03-12 08:03:00|  AAPL|            

In [125]:
sma10Win = Window.partitionBy("Ticker","Date").orderBy("datetime").rangeBetween(-10,0)
df = df.withColumn('sma10',mean(df.LastTradePrice).over(sma10Win))

VBox()

An error was encountered:
Invalid status code '404' from http://localhost:8998/sessions/1 with error payload: {"msg":"Session '1' not found."}


In [39]:
apl = df.filter(df.Ticker =="AAPL")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
sample = df.sample(fraction=0.0001)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
features = ['ret1min', 'ret2min', 'ret3min', 'ret4min', 'ret5min', 
            'ret6min', 'ret7min', 'ret8min', 'ret9min', 'ret10min','cci','mfi']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Add indicators and features

In [91]:
# import finratios

ratio = spark.read.parquet('s3://securityfundamentalsdata/finratio_processed.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
ratio.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Unnamed:_0: long (nullable = true)
 |-- gvkey: string (nullable = true)
 |-- permno: double (nullable = true)
 |-- adate: timestamp (nullable = true)
 |-- qdate: timestamp (nullable = true)
 |-- public_date: timestamp (nullable = true)
 |-- CAPEI: double (nullable = true)
 |-- BE: double (nullable = true)
 |-- bm: double (nullable = true)
 |-- evm: double (nullable = true)
 |-- pe_op_basic: double (nullable = true)
 |-- pe_op_dil: double (nullable = true)
 |-- pe_exi: double (nullable = true)
 |-- pe_inc: double (nullable = true)
 |-- ps: double (nullable = true)
 |-- pcf: double (nullable = true)
 |-- dpr: double (nullable = true)
 |-- npm: double (nullable = true)
 |-- opmbd: double (nullable = true)
 |-- opmad: double (nullable = true)
 |-- gpm: double (nullable = true)
 |-- ptpm: double (nullable = true)
 |-- cfm: double (nullable = true)
 |-- roa: double (nullable = true)
 |-- roe: double (nullable = true)
 |-- roce: double (nullable = true)
 |-- efftax: double (nullable

In [93]:
ratio.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------+-------------------+-------------------+-------------------+------------------+------+-------------------+------------------+-----------+---------+------------------+-------------------+-------------------+------------------+---+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+-------------------+------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+--------------------+-------------------+------------------+------------------+-------------------+------------------+----

In [94]:
apl.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------------+------------+-----------+------------+-----------+------------------+---------------+--------------+------------------+------------+-----------+------------------+------------+-----------+------------------+--------------+-------------+------------------+-----------+----------+------------------+-----------+----------+------------------+-------------+------------+------------------+-------------+------------+-------------+------------+------------------+--------------+-------------+---------+---------+----------+-----------------+--------------+----------+-------------+----------+-------------+----------+--------------------+------+-----------+-----------+----------------------+------------+--------------+------------------+--------------------+-----------------+-------------------+---------------------------+-------------+-------------+-------------------+------+----------+-----------+-----------------------+
|TimeBarStart|       OpenBarTime|OpenBidPri

In [104]:
apl = apl.join(ratio, (apl.Ticker==ratio.TICKER) &(apl.Date==ratio.date),"cross")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
apl.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- TimeBarStart: string (nullable = true)
 |-- OpenBarTime: string (nullable = true)
 |-- OpenBidPrice: double (nullable = true)
 |-- OpenBidSize: integer (nullable = true)
 |-- OpenAskPrice: double (nullable = true)
 |-- OpenAskSize: integer (nullable = true)
 |-- FirstTradeTime: string (nullable = true)
 |-- FirstTradePrice: double (nullable = true)
 |-- FirstTradeSize: integer (nullable = true)
 |-- HighBidTime: string (nullable = true)
 |-- HighBidPrice: double (nullable = true)
 |-- HighBidSize: integer (nullable = true)
 |-- HighAskTime: string (nullable = true)
 |-- HighAskPrice: double (nullable = true)
 |-- HighAskSize: integer (nullable = true)
 |-- HighTradeTime: string (nullable = true)
 |-- HighTradePrice: double (nullable = true)
 |-- HighTradeSize: integer (nullable = true)
 |-- LowBidTime: string (nullable = true)
 |-- LowBidPrice: double (nullable = true)
 |-- LowBidSize: integer (nullable = true)
 |-- LowAskTime: string (nullable = true)
 |-- LowAskPrice: doubl

In [106]:
apl.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------------+------------+-----------+------------+-----------+------------------+---------------+--------------+------------------+------------+-----------+------------------+------------+-----------+------------------+--------------+-------------+------------------+-----------+----------+------------------+-----------+----------+------------------+-------------+------------+------------------+-------------+------------+-------------+------------+------------------+--------------+-------------+---------+---------+----------+-----------------+--------------+----------+-------------+----------+-------------+----------+--------------------+------+-----------+-----------+----------------------+------------+--------------+------------------+--------------------+-----------------+-------------------+---------------------------+-------------+-------------+-------------------+------+----------+--------------------+-----------------------+----------+------+-------+---------

In [112]:
aapl = apl.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
import awswrangler as wr
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [114]:
wr.s3.to_csv(aapl, 's3://strategymodeldata/tstAaplTaqRatio.csv',index=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'paths': ['s3://strategymodeldata/tstAaplTaqRatio.csv'], 'partitions_values': {}}

In [118]:
aapl.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  TimeBarStart         OpenBarTime  OpenBidPrice  OpenBidSize  OpenAskPrice  \
0        04:00  04:00:00.000000000           NaN          NaN           NaN   
1        04:01  04:01:00.000000000        166.22        100.0        166.95   
2        04:02  04:02:00.000000000        166.21        100.0        166.98   
3        04:03  04:03:00.000000000        166.22        400.0        166.96   
4        04:04  04:04:00.000000000        166.22        300.0        166.88   

   OpenAskSize      FirstTradeTime  FirstTradePrice  FirstTradeSize  \
0          NaN  04:00:00.018908177           166.35            10.0   
1        100.0  04:01:07.997355535           166.41            20.0   
2       1200.0  04:02:07.092574587           166.78             5.0   
3        100.0  04:03:57.943770734           166.41            30.0   
4       1000.0  04:04:38.380804376           166.62            10.0   

          HighBidTime  HighBidPrice  HighBidSize         HighAskTime  \
0  04:00:00.083360119     

In [124]:
aapl.info(null_counts=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33602 entries, 0 to 33601
Columns: 166 entries, TimeBarStart to date
dtypes: datetime64[ns](4), float64(113), int32(21), int64(1), object(27)
memory usage: 39.9+ MB
<string>:1: FutureWarning: null_counts is deprecated. Use show_counts instead

## SMA

In [ ]:
def get_sma(X, df):
    """
    Get features related to simple moving average
    Inputs
        X  : dataframe without the SMA features
        df : dataframe containing 'adj_close' column. To generate features for the last row in this dataframe.
    Outputs
        X  : X with additional SMA features
    """
    
    # Simple moving average
    df['sma_10'] = df['adj_close'].rolling(window = 10).mean()
    df['sma_20'] = df['adj_close'].rolling(window = 20).mean()
    df['sma_50'] = df['adj_close'].rolling(window = 50).mean()
    df['sma_100'] = df['adj_close'].rolling(window = 100).mean()
    df['sma_200'] = df['adj_close'].rolling(window = 200).mean()

    # Get the sma on the previous day
    df['adj_close_lag_1'] = df['adj_close'].shift(1)
    df['sma_10_lag_1'] = df['sma_10'].shift(1)
    df['sma_20_lag_1'] = df['sma_20'].shift(1)
    df['sma_50_lag_1'] = df['sma_50'].shift(1)
    df['sma_100_lag_1'] = df['sma_100'].shift(1)
    df['sma_200_lag_1'] = df['sma_200'].shift(1)

    # Get above signals
    df['sma_above20'] = (df['adj_close'] > df['sma_20']).astype(int)
    df['sma_above50'] = (df['adj_close'] > df['sma_50']).astype(int)
    df['sma_above100'] = (df['adj_close'] > df['sma_100']).astype(int)
    df['sma_above200'] = (df['adj_close'] > df['sma_200']).astype(int)

    df['sma_10above20'] = (df['sma_10'] > df['sma_20']).astype(int)
    df['sma_10above50'] = (df['sma_10'] > df['sma_50']).astype(int)
    df['sma_10above100'] = (df['sma_10'] > df['sma_100']).astype(int)
    df['sma_10above200'] = (df['sma_10'] > df['sma_200']).astype(int)

    # Get bullish crossover signals
    df['sma_cut20'] = ((df['adj_close_lag_1'] < df['sma_20_lag_1']) & (df['sma_above20']==True)).astype(int)
    df['sma_cut50'] = ((df['adj_close_lag_1'] < df['sma_50_lag_1']) & (df['sma_above50']==True)).astype(int)
    df['sma_cut100'] = ((df['adj_close_lag_1'] < df['sma_100_lag_1']) & (df['sma_above100']==True)).astype(int)
    df['sma_cut200'] = ((df['adj_close_lag_1'] < df['sma_200_lag_1']) & (df['sma_above200']==True)).astype(int)

    df['sma_10cut20'] = ((df['sma_10_lag_1'] < df['sma_20_lag_1']) & (df['sma_10above20']==True)).astype(int)
    df['sma_10cut50'] = ((df['sma_10_lag_1'] < df['sma_50_lag_1']) & (df['sma_10above50']==True)).astype(int)
    df['sma_10cut100'] = ((df['sma_10_lag_1'] < df['sma_100_lag_1']) & (df['sma_10above100']==True)).astype(int)
    df['sma_10cut200'] = ((df['sma_10_lag_1'] < df['sma_200_lag_1']) & (df['sma_10above200']==True)).astype(int)

    # Get bearish crossover signals
    df['sma_cut20down'] = ((df['adj_close_lag_1'] > df['sma_20_lag_1']) & (df['adj_close'] < df['sma_20'])).astype(int)
    df['sma_cut50down'] = ((df['adj_close_lag_1'] > df['sma_50_lag_1']) & (df['adj_close'] < df['sma_50'])).astype(int)
    df['sma_cut100down'] = ((df['adj_close_lag_1'] > df['sma_100_lag_1']) & (df['adj_close'] < df['sma_100'])).astype(int)
    df['sma_cut200down'] = ((df['adj_close_lag_1'] > df['sma_200_lag_1']) & (df['adj_close'] < df['sma_200'])).astype(int)

    df['sma_10cut20down'] = ((df['sma_10_lag_1'] > df['sma_20_lag_1']) & (df['sma_10'] < df['sma_20'])).astype(int)
    df['sma_10cut50down'] = ((df['sma_10_lag_1'] > df['sma_50_lag_1']) & (df['sma_10'] < df['sma_50'])).astype(int)
    df['sma_10cut100down'] = ((df['sma_10_lag_1'] > df['sma_100_lag_1']) & (df['sma_10'] < df['sma_100'])).astype(int)
    df['sma_10cut200down'] = ((df['sma_10_lag_1'] > df['sma_200_lag_1']) & (df['sma_10'] < df['sma_200'])).astype(int)
    
    tech_features = [   'sma_above20',
                        'sma_above50',
                        'sma_above100',
                        'sma_above200',
                        'sma_10above20',
                        'sma_10above50',
                        'sma_10above100',
                        'sma_10above200',
                        'sma_cut20',
                        'sma_cut50',
                        'sma_cut100',
                        'sma_cut200',
                        'sma_10cut20',
                        'sma_10cut50',
                        'sma_10cut100',
                        'sma_10cut200',
                        'sma_cut20down',
                        'sma_cut50down',
                        'sma_cut100down',
                        'sma_cut200down',
                        'sma_10cut20down',
                        'sma_10cut50down',
                        'sma_10cut100down',
                        'sma_10cut200down'
                    ]
    
    features_df = df[tech_features][-1:].reset_index(drop=True)
    features_df.columns = [item+'_lag_1' for item in tech_features]
    
    return pd.concat([X, features_df], axis=1)

## 

## XGBoost SageMaker PySpark

In [ ]:
from pyspark import SparkContext, SparkConf
from sagemaker_pyspark import IAMRole, classpath_jars
from sagemaker_pyspark.algorithms import XGBoostSageMakerEstimator

# Load the sagemaker_pyspark classpath. If you used --jars to submit your job
# there is no need to do this in code.
conf = (SparkConf()
        .set("spark.driver.extraClassPath", ":".join(classpath_jars())))
SparkContext.getOrCreate(conf=conf)

#iam_role = "arn:aws:iam:0123456789012:role/MySageMakerRole"

region = "us-east-1"
training_data = (spark.read.format("libsvm").option("numFeatures", "784")
  .load("s3a://sagemaker-sample-data-{}/spark/mnist/train/".format(region)))

test_data = (spark.read.format("libsvm").option("numFeatures", "784")
  .load("s3a://sagemaker-sample-data-{}/spark/mnist/train/".format(region)))

xgboost_estimator = XGBoostSageMakerEstimator(
    trainingInstanceType="ml.m4.xlarge",
    trainingInstanceCount=1,
    endpointInstanceType="ml.m4.xlarge",
    endpointInitialInstanceCount=1,
    sagemakerRole=IAMRole(iam_role))

xgboost_estimator.setObjective('multi:softmax')
xgboost_estimator.setNumRound(25)
xgboost_estimator.setNumClasses(10)

xgboost_model = xgboost_estimator.fit(training_data)

transformed_data = xgboost_model.transform(test_data)
transformed_data.show()